In [1]:
import pyOSC3
import time
import random
import copy
from collections import namedtuple, defaultdict
from subprocess import Popen
import numpy as np
import math
import json

from chord_utils import *
from kriste_aghsdga_chords import *

In [2]:
client = pyOSC3.OSCClient()
client.connect(('127.0.0.1', 57120))

In [3]:
chords = get_chords()

# [1701, 1843, 2024, 2222, 2424, 2550, 2741, 2958]
redux_cents = [1701, 1843, 2024, 2228, 2432, 2543, 2747, 2951] +\
[2136, 3010, 3090]
#[1512, 1667, 1857, 2057, 2247, 2387, 2429, 2547, 2747, 2887] +\
#[2347, 2687] + [2057+30, 2547+30, 2747+30]

# check_chords(chords, redux_cents)


In [20]:
# proc = Popen(['afplay', '/Users/a1111/Downloads/GVM097_KristeAghsdga_MVCM.mp3'])


def send_msg(note, cur_timbre, out_bus_idx):
    try:
        msg = pyOSC3.OSCMessage()
        msg.setAddress("/synths")

        msg.append([note, cur_timbre, out_bus_idx])
        # print('sent', note,'to', out_bus_idx)
        client.send(msg)
#         time.sleep(0.001)
    except Exception as e:
        pass

ref_hz = 110

prev_time = 0
sleep_time = chords[0].timestamp
# time.sleep(sleep_time)

for idx, chord in enumerate(chords):
    if idx < 72:
        continue
    if idx + 1 < len(chords):
        sleep_time = chords[idx + 1].timestamp - chords[idx].timestamp
    else:
        sleep_time = 2

    print(idx, chord.syllable, chord.chord, chord.redux)
    hzs = [calc_hz(pitch, ref_hz) for pitch in chord.chord]
    r_hzs = [calc_hz(pitch, ref_hz) for pitch in chord.redux]
    for hz in hzs:
        send_msg(hz, 0, 0)
    time.sleep(sleep_time)
#     continue
#     engine.play_chord(hzs, notes_to_compare=r_hzs, duration=sleep_time, loudness=3, pan=0.0)
#     engine.play_chord(r_hzs, duration=sleep_time, loudness=3, pan=0.5)
#     engine.play_chord(hzs, duration=sleep_time, loudness=3, pan=0.5)

# proc.terminate()


72 kri [2035, 2549, 2939] [2024, 2543, 2951]
73 ste [1991, 2549, 2942] [2024, 2543, 2951]
74 aghs [2000, None, 2936] [2024, None, 2951]
75 dga [2020, 2550, 2948] [2024, 2543, 2951]
76 mkvdre [2025, 2728, 2743] [2024, 2747, 2747]
77 tit [2013, 2428, 2711] [2024, 2432, 2747]
78 si [2118, 2155, 2864] [2228, 2228, 2951]
79 kvdi [2226, 2232, 2921] [2228, 2228, 2951]
80 li [2232, 2241, 2927] [2228, 2228, 2951]
81 ta [2197, 2199, 2935] [2228, 2228, 2951]
82 si [2188, 2672, 2885] [2228, 2747, 2951]
83 kvdi [2209, 2736, 2928] [2228, 2747, 2951]
84 li [2203, 2737, 2923] [2228, 2747, 2951]
85 sa/ta [1793, 2509, 2936] [1843, 2543, 2951]
86 i [None, None, 2529] [None, None, 2951]
87 da [1997, 2498, 2740] [2024, 2543, 2747]
88 trgun [2020, 2512, 2738] [2024, 2543, 2747]
89 ve [2007, 2504, 2995] [2024, 2543, 3090]
90 li [2012, 2515, 2943] [2024, 2543, 2951]
91 da [2013, 2513, 2784] [2024, 2543, 2747]
92 sa [2035, 2506, 2532] [2024, 2543, 2543]
93 pla [2013, 2518, 2761] [2024, 2543, 2747]
94 ve [1958,

In [21]:
def normalize(r):
    n = [r[0], r[1]]
    while 2 <= n[0] / n[1]:
        n[1] *= 2
    while n[0] / n[1] <= 1:
        n[0] *= 2
    return tuple(n)

# wilson41
# crystal growth algorithm
primes = [3, 5, 7, 11, 13]
ratios = [(1, 1), (2, 1)]
for i, p in enumerate(primes):
    if p == 13:
        break
    r1 = [p, 1]
    max_deg = len(primes) - i
    if p == 3:
        max_deg -= 1
    for j in range(max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[0] *= 3
    if p == 3:
        continue
    r1 = [p, 3]
    for j in range(1, max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[1] *= 3

odds = [1, 3, 5, 7, 9, 11]
for p1 in odds:
    for p2 in odds:
        if p1 == p2:
            continue
        if min(p1, p2) != 1 and (max(p1, p2) % min(p1, p2) == 0):
            continue
        # wilson example
        # if (p1, p2) in [(11, 5), (5, 11)]:
            # continue
        r = (p1, p2)
        r = normalize(r)
        if r not in ratios:
            ratios.append(r)

# ratios.sort(key=lambda r: util.ratio_to_cents(r[0], r[1]))
print('ratios count', len(ratios))


ratios count 48


In [28]:
cents_tolerance = 100 / 8

ref_hz = 55

main_pitch_in_cents = 2024
main_pitch_in_hz = calc_hz(main_pitch_in_cents, ref_hz)

ret_sort = sorted(redux_cents)
ret_sort = np.array(ret_sort) - main_pitch_in_cents
print(ret_sort)

intervals = [defaultdict(int), defaultdict(int), defaultdict(int)]
intervals_as_pairs = [dict(), dict(), dict()]
intervals_as_ji = dict()

all_intervals = defaultdict(int)
all_intervals_as_pairs = dict()

cents_count = [defaultdict(int), defaultdict(int), defaultdict(int)]
all_cents_count = defaultdict(int)

all_chords = defaultdict(int)
for chord in chords:
    pitches = chord.redux
    pitches_for_diff = []
    pitches_in_hz = []
    pitches_in_cents = []
    pitches_in_cents_with_none = []
    for p_idx, pitch in enumerate(pitches):
        if pitch is None:
            pitches_in_cents_with_none.append(None)
            continue
        cents_count[p_idx][pitch] += 1
        all_cents_count[pitch] += 1
        pitches_for_diff.append(pitch)
        pitch_in_hz = calc_hz(pitch, main_pitch_in_hz)
        pitches_in_hz.append(pitch_in_hz / main_pitch_in_hz)
        pitch_in_cents = int(ratio_to_cents(pitch_in_hz, main_pitch_in_hz))
        pitches_in_cents.append(pitch_in_cents)
        pitches_in_cents_with_none.append(pitch_in_cents)

    # chord intervals
    if len(pitches_for_diff) == 1:
        diffs = []
    elif len(pitches_for_diff) == 2:
        diffs = [pitches_for_diff[1] - pitches_for_diff[0]]
    else:
        diffs = [pitches_for_diff[2] - pitches_for_diff[0],
                 pitches_for_diff[2] - pitches_for_diff[1],
                 pitches_for_diff[1] - pitches_for_diff[0]]
        pairs = [(pitches_for_diff[0], pitches_for_diff[2]),
                 (pitches_for_diff[1], pitches_for_diff[2]),
                 (pitches_for_diff[0], pitches_for_diff[1])]

        for k in range(3):
            intervals[k][diffs[k]] += 1
            all_intervals[diffs[k]] += 1
            if diffs[k] not in intervals_as_pairs[k]:
                intervals_as_pairs[k][diffs[k]] = set()
            if diffs[k] not in all_intervals_as_pairs:
                all_intervals_as_pairs[diffs[k]] = set()
            intervals_as_pairs[k][diffs[k]].add(pairs[k])
            all_intervals_as_pairs[diffs[k]].add(pairs[k])
        ch = tuple(diffs)
        all_chords[ch] += 1

    diffs = [abs(c) for c in diffs]

    notes = []
    for i, p in enumerate(diffs):
        n = str(diffs[i]) + 'c'
        state = ''
        for r in ratios:
            c1 = diffs[i]
            c2 = ratio_to_cents(r[0], r[1])
            check1 = abs(c1 - c2) < cents_tolerance
            check2 = abs(c1 - c2) < cents_tolerance * 1.5
            add = ''
            if check2 and not check1:
                add = '?'
            if check2:
                if r == (1, 1):
                    state = '.' + add
                else:
                    state = 'ji'
                    n += ' ' + add + str(r[0]) + '/' + str(r[1])
        notes.append((n, state))
#     print(chord.syllable, pitches, notes)

    if len(diffs) == 3:
        for k in range(3):
            pretty = ' '.join(notes[k][0].split(' ')[1:])
            intervals_as_ji[diffs[k]] = pretty + ' ' + notes[k][1]

for i in range(3):
    print(i)
    for k in sorted(intervals[i].keys()):
        print(k, intervals[i][k], intervals_as_pairs[i][k], intervals_as_ji[abs(k)])
    print()

print('all intervals')
for k in sorted(all_intervals.keys()):
    print(k, all_intervals[k], all_intervals_as_pairs[k], intervals_as_ji[abs(k)])
print()

    
print('cents counts')
for i in range(3):
    print(i)
    for k in sorted(cents_count[i].keys()):
        print(k, cents_count[i][k])
    print()

print('all cents')
for k in sorted(all_cents_count.keys()):
    print(k, all_cents_count[k])

print('all chords')
for ch in all_chords:
    print(all_chords[ch], ch)

[-323 -181    0  204  408  519  723  927 1066]
0
408 2 {(2024, 2432)} 81/64 14/11 ji
519 7 {(2024, 2543)} 27/20 ji
589 3 {(1843, 2432)} 45/32 7/5 ji
700 4 {(1843, 2543)} 3/2 ji
723 49 {(2228, 2951), (2024, 2747)} 32/21 ji
927 16 {(2024, 2951)} 12/7 ji
1066 6 {(2024, 3090)} ?11/6 ji
1108 6 {(1843, 2951)} 256/135 ji
1247 3 {(1843, 3090)}  

1
-204 1 {(2747, 2543)} 9/8 ji
0 14 {(2747, 2747), (2543, 2543), (2432, 2432)}  .
204 30 {(2747, 2951), (2543, 2747)} 9/8 ji
315 5 {(2228, 2543), (2432, 2747)} 6/5 ji
408 22 {(2543, 2951)} 81/64 14/11 ji
547 9 {(2543, 3090)} 11/8 ji
723 15 {(2228, 2951), (2024, 2747)} 32/21 ji

2
0 15 {(2024, 2024), (2228, 2228)}  .
315 3 {(2228, 2543)} 6/5 ji
385 3 {(1843, 2228)} 5/4 ji
408 4 {(2024, 2432)} 81/64 14/11 ji
519 52 {(2024, 2543), (2228, 2747)} 27/20 ji
589 3 {(1843, 2432)} 45/32 7/5 ji
700 10 {(1843, 2543)} 3/2 ji
723 6 {(2024, 2747)} 32/21 ji

all intervals
-204 1 {(2747, 2543)} 9/8 ji
0 29 {(2747, 2747), (2432, 2432), (2024, 2024), (2543, 2543), (2228

In [26]:
print(sorted(redux_cents))

[1701, 1843, 2024, 2228, 2432, 2543, 2747, 2951, 3090]
